In [1]:
import tvm 
import numpy as np

## Describe Sum of Rows

`B = numpy.sum(A, axis=1)`

In [2]:
n = tvm.var("n")
m = tvm.var("m")

In [3]:
A = tvm.placeholder((n, m), name="A")
k = tvm.reduce_axis(dom=(0, m), name="k")
B = tvm.compute(shape=(n,), fcompute=lambda i: tvm.sum(A[i, k], axis=k), name="B")

## Schedule the Reduction

In [4]:
s = tvm.create_schedule(B.op)
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

produce B {
  for (i, 0, n) {
    B[i] = 0.000000f
    for (k, 0, m) {
      B[i] = (B[i] + A[((i*m) + k)])
    }
  }
}



Let's split both the row axis of B as well axis by different factors. The result is a nested reduction. 

In [6]:
ko, ki = s[B].split(parent=B.op.reduce_axis[0], factor=16)
xo, xi = s[B].split(parent=B.op.axis[0], factor=32)

In [7]:
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

produce B {
  for (i.outer, 0, ((n + 31)/32)) {
    for (i.inner, 0, 32) {
      if (likely(((i.outer*32) < (n - i.inner)))) {
        B[((i.outer*32) + i.inner)] = 0.000000f
      }
      for (k.outer, 0, ((m + 15)/16)) {
        for (k.inner, 0, 16) {
          if (likely(((i.outer*32) < (n - i.inner)))) {
            if (likely(((k.outer*16) < (m - k.inner)))) {
              B[((i.outer*32) + i.inner)] = (B[((i.outer*32) + i.inner)] + A[(((((i.outer*32) + i.inner)*m) + (k.outer*16)) + k.inner)])
            }
          }
        }
      }
    }
  }
}



If we are building a GPU kernel, we can bind the rows of B to GPU threads

In [8]:
s[B].bind(ivar=xo, thread_ivar=tvm.thread_axis("blockIdx.x"))
s[B].bind(ivar=xi, thread_ivar=tvm.thread_axis("threadIdx.x"))
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

produce B {
  // attr [iter_var(blockIdx.x, , blockIdx.x)] thread_extent = ((n + 31)/32)
  // attr [iter_var(threadIdx.x, , threadIdx.x)] thread_extent = 32
  if (likely(((blockIdx.x*32) < (n - threadIdx.x)))) {
    B[((blockIdx.x*32) + threadIdx.x)] = 0.000000f
  }
  for (k.outer, 0, ((m + 15)/16)) {
    for (k.inner, 0, 16) {
      if (likely(((blockIdx.x*32) < (n - threadIdx.x)))) {
        if (likely(((k.outer*16) < (m - k.inner)))) {
          B[((blockIdx.x*32) + threadIdx.x)] = (B[((blockIdx.x*32) + threadIdx.x)] + A[(((((blockIdx.x*32) + threadIdx.x)*m) + (k.outer*16)) + k.inner)])
        }
      }
    }
  }
}



## Reduction Factoring and Parallelization

In [9]:
s = tvm.create_schedule(ops=B.op)

In [10]:
ko, ki = s[B].split(parent=B.op.reduce_axis[0], factor=16)

In [11]:
BF = s.rfactor(tensor=B, axis=ki, factor_axis=0)

In [12]:
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

// attr [B.rf] storage_scope = "global"
allocate B.rf[float32 * 16 * n]
produce B.rf {
  for (k.inner, 0, 16) {
    for (i, 0, n) {
      B.rf[((k.inner*n) + i)] = 0.000000f
      for (k.outer, 0, ((m + 15)/16)) {
        if ((k.inner < (m - (k.outer*16)))) {
          B.rf[((k.inner*n) + i)] = (B.rf[((k.inner*n) + i)] + A[((k.inner + (i*m)) + (k.outer*16))])
        }
      }
    }
  }
}
produce B {
  for (ax0, 0, n) {
    B[ax0] = 0.000000f
    for (k.inner.v, 0, 16) {
      B[ax0] = (B[ax0] + B.rf[(ax0 + (k.inner.v*n))])
    }
  }
}



In [14]:
print(s[B].op.body)

[reduce(combiner=comm_reducer(result=[(x + y)], lhs=[x], rhs=[y], identity_element=[0.000000f]), source=[B.rf(k.inner.v, ax0)], axis=[iter_var(k.inner.v, Range(min=0, extent=16))], where=(uint1)1, value_index=0)]


## Cross Thread Reduction 

- We can now parallelize over the factored axis. 
- Here the reduction axis of B is marked to be a thread.
- TVM allows reduction axis to be marked as thread if it is the only axis in reduction and cross thread reduction is possible in the device. 

In [15]:
xo, xi = s[B].split(s[B].op.axis[0], factor=32)
s[B].bind(xo, tvm.thread_axis("blockIdx.x"))
s[B].bind(xi, tvm.thread_axis("threadIdx.y"))
tx = tvm.thread_axis("threadIdx.x")
s[B].bind(s[B].op.reduce_axis[0], tx)
s[BF].compute_at(s[B], s[B].op.reduce_axis[0])
s[B].set_store_predicate(tx.var.equal(0))
fcuda = tvm.build(s, [A, B], "cuda")
print(fcuda.imported_modules[0].get_source())

extern "C" __global__ void default_function__kernel0( float* __restrict__ A,  float* __restrict__ B, int m, int n) {
   float B_rf[1];
  __shared__ float red_buf0[512];
  B_rf[0] = 0.000000e+00f;
  for (int k_outer = 0; k_outer < ((15 + m) / 16); ++k_outer) {
    if ((((int)blockIdx.x) * 32) < (n - ((int)threadIdx.y))) {
      if (((int)threadIdx.x) < (m - (k_outer * 16))) {
        B_rf[0] = (B_rf[0] + A[(((((((int)blockIdx.x) * 32) + ((int)threadIdx.y)) * m) + ((int)threadIdx.x)) + (k_outer * 16))]);
      }
    }
  }
  ((volatile __shared__ float*)red_buf0)[((((int)threadIdx.y) * 16) + ((int)threadIdx.x))] = (((((int)blockIdx.x) * 32) < (n - ((int)threadIdx.y))) ? B_rf[0] : 0.000000e+00f);
  __syncthreads();
  if (((int)threadIdx.x) < 8) {
    ((volatile __shared__ float*)red_buf0)[((((int)threadIdx.y) * 16) + ((int)threadIdx.x))] = (((volatile __shared__ float*)red_buf0)[((((int)threadIdx.y) * 16) + ((int)threadIdx.x))] + ((volatile __shared__ float*)red_buf0)[((8 + (((int)threadId

In [16]:
nn = 128
ctx  = tvm.gpu(0)
a = tvm.nd.array(np.random.uniform(size=(nn, nn)).astype(A.dtype), ctx)
b = tvm.nd.array(np.zeros(nn, dtype=B.dtype), ctx)
fcuda(a, b)

In [17]:
np.testing.assert_allclose(b.asnumpy(), np.sum(a.asnumpy(), axis=1), rtol=1e-4)

## Describe Convolution via 2D Reduction 

In [18]:
n = tvm.var("n")
Input = tvm.placeholder(shape=(n, n), name="Input")
Filter = tvm.placeholder(shape=(3, 3), name="Filter")

In [19]:
di = tvm.reduce_axis(dom=(0, 3), name="di")
dj = tvm.reduce_axis(dom=(0, 3), name="dj")

In [20]:
Output = tvm.compute(shape=(n-2, n-2),
                    fcompute=lambda i, j: tvm.sum(Input[i + di, j + dj] * Filter[di, dj], axis=[di, dj]),
                    name="Output")

In [21]:
s = tvm.create_schedule(ops=Output.op)

In [22]:
print(tvm.lower(sch=s, args=[Input, Filter, Output], simple_mode=True))

produce Output {
  for (i, 0, (n + -2)) {
    for (j, 0, (n + -2)) {
      Output[((i*(n + -2)) + j)] = 0.000000f
      for (di, 0, 3) {
        for (dj, 0, 3) {
          Output[((i*(n + -2)) + j)] = (Output[((i*(n + -2)) + j)] + (Input[((j + ((i + di)*n)) + dj)]*Filter[((di*3) + dj)]))
        }
      }
    }
  }
}



## Define General Commutative Reduction Operation 

In [23]:
n = tvm.var('n')
m = tvm.var('m')
product = tvm.comm_reducer(lambda x, y: x*y,
    lambda t: tvm.const(1, dtype=t), name="product")
A = tvm.placeholder((n, m), name='A')
k = tvm.reduce_axis((0, m), name='k')
B = tvm.compute((n,), lambda i: product(A[i, k], axis=k), name='B')

In [24]:
s = tvm.create_schedule(ops=B.op)

In [26]:
print(tvm.lower(sch=s, args=[A, B], simple_mode=True))

produce B {
  for (i, 0, n) {
    B[i] = 1.000000f
    for (k, 0, m) {
      B[i] = (B[i]*A[((i*m) + k)])
    }
  }
}

